In [1]:
import sys
!{sys.executable} -m pip install opencv-python

In [89]:
%matplotlib qt
import os
import cv2
import scipy.ndimage
import numpy as np

class NoiseRemover():
    def _erase_circles(img, circles):
        circles = circles[0] # hough circles returns a nested list for some reason
        for circle in circles:
            x = circle[0] # x coordinate of circle's center
            y = circle[1] # y coordinate of circle's center
            r = circle[2] # radius of circle

            img = cv2.circle(img, (int(x), int(y)), radius = int(r), color = (255,0,0), thickness = 2) # erase circle by making it white (to match the image background)
        return img


    def _detect_and_remove_circles(img):
        hough_circle_locations = cv2.HoughCircles(img, method = cv2.HOUGH_GRADIENT, dp = 1, minDist = 1, param1 = 50, param2 = 5, minRadius = 0, maxRadius = 2)
        if hough_circle_locations is not None:
            img = NoiseRemover._erase_circles(img, hough_circle_locations)
            pass
        return img

    def remove_all_noise(img):
        # run some basic tests to get rid of easy-to-remove noise -- first pass
        img = ~img # white letters, black background
        img = cv2.erode(img, np.ones((3, 2), np.uint8), iterations = 1) # weaken circle noise and line noise
        img = ~img # black letters, white background
        img = scipy.ndimage.median_filter(img, (5, 1)) # remove line noise
        img = scipy.ndimage.median_filter(img, (1, 3)) # weaken circle noise
        img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations = 1) # dilate image to initial stage (erode works similar to dilate because we thresholded the image the opposite way)
        img = scipy.ndimage.median_filter(img, (3, 3)) # remove any final 'weak' noise that might be present (line or circle)

#         detect any remaining circle noise
        img = NoiseRemover._detect_and_remove_circles(img) # after dilation, if concrete circles exist, use hough transform to remove them
#         show(img)
#         eradicate any final noise that wasn't removed previously -- second pass
        img = cv2.dilate(img, np.ones((3, 3), np.uint8), iterations = 1) # actually performs erosion
        img = scipy.ndimage.median_filter(img, (3, 1)) # finally completely remove any extra noise that remains
        img = cv2.dilate(img, np.ones((2, 2), np.uint8), iterations = 1) # erode just a bit to polish fine details
        img = cv2.erode(img, np.ones((2, 2), np.uint8), iterations = 2) # dilate image to make it look like the original
        return img

#############################

import cv2
import numpy as np

def show(img):
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()    

# img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# MgP48, 

img_o = cv2.imread("train/MgP48.png",cv2.IMREAD_GRAYSCALE)
# img_o = cv2.cvtColor(img_o, cv2.COLOR_BGR2RGB)
_, img = cv2.threshold(img_o, 222, 255, cv2.THRESH_BINARY) # binarize the image

d_img = NoiseRemover.remove_all_noise(img)
print(d_img)

d_img = cv2.cvtColor(d_img, cv2.COLOR_BGR2RGB)


import numpy as np
import cv2
from matplotlib import pyplot as plt
plt.subplot(211),plt.imshow(img_o)
plt.subplot(212),
plt.imshow(d_img)
plt.show()

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
